# Wstęp do sieci neuronowych


<img src="figures/L9/cat_neuron.jpg" width=500>

Ref: 

* http://cs231n.github.io/optimization-1/ 

* https://github.com/leriomaggio/deep-learning-keras-tensorflow/blob/master/1.3%20Introduction%20-%20Keras.ipynb

* http://colah.github.io/posts/2014-10-Visualizing-MNIST/

* https://github.com/peterroelants/peterroelants.github.io/blob/master/notebooks/neural_net_implementation/neural_network_implementation_part02.ipynb


Na dzisiejszych zajęciach omówimy jak można zaimplementować jako sieć neuronową znany nam model regresji logistycznej, najpierw bardziej ręcznie, a następnie z użyciem pakietu keras. Omówimy na końcu spojrzenie na sieci neuronowe z perspektywy uczenia reprezentacji.

# Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from sklearn.decomposition import RandomizedPCA
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle
import matplotlib.pyplot as plt 
from matplotlib.colors import colorConverter, ListedColormap
from matplotlib import cm #
import os
import tqdm
os.environ['KERAS_BACKEND'] = 'theano'
import keras
from keras.datasets import mnist
from sklearn.datasets import load_iris
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
%matplotlib inline
np.random.seed(seed=1)

ModuleNotFoundError: No module named 'keras'

In [ ]:
# Small MNIST 
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()
sm_train = mnist_x_train[0:500].astype('float32') / 255.
sm_y_train = mnist_y_train[0:500]
sm_test = mnist_x_test[0:500].astype('float32') / 255.
sm_y_test = mnist_y_test[0:500]
sm_train = sm_train.reshape((len(sm_train), np.prod(sm_train.shape[1:])))
sm_test = sm_test.reshape((len(sm_test), np.prod(sm_test.shape[1:])))
print sm_train.shape
print sm_test.shape

In [ ]:
if not keras.__version__[0] == '2':
    raise Exception("Prosze zainstalowac keras>=2.0.0")

In [ ]:
def logistic(z): 
    return 1 / (1 + np.exp(-z))

def nn(x, w): 
    return logistic(x.dot(w.T))

def nn_predict(x,w): 
    return np.around(nn(x,w))

def cost(y, t):
    return - np.sum(np.multiply(t, np.log(y)) + np.multiply((1-t), np.log(1-y)))

In [ ]:
# Define and generate the samples
nb_of_samples_per_class = 100  # The number of sample in each class
red_mean = [-1,0]  # The mean of the red class
blue_mean = [1,0]  # The mean of the blue class
std_dev = 1.2  # standard deviation of both classes
# Generate samples from both classes
x_red = np.random.randn(nb_of_samples_per_class, 2) * std_dev + red_mean
x_blue = np.random.randn(nb_of_samples_per_class, 2) * std_dev + blue_mean

# Merge samples in set of input variables x, and corresponding set of output variables t
X_toy = np.vstack((x_red, x_blue))
t_toy = np.vstack((np.zeros((nb_of_samples_per_class,1)), np.ones((nb_of_samples_per_class,1))))

toy_train, toy_test, toy_y_train, toy_y_test = train_test_split(X_toy, t_toy, test_size=0.15)

# Przyda sie do sekcji o kerasie
toy_y_train_one_hot = keras.utils.to_categorical(toy_y_train)
toy_y_test_one_hot = keras.utils.to_categorical(toy_y_test)

In [ ]:

# Plot both classes on the x1, x2 plane
plt.plot(x_red[:,0], x_red[:,1], 'ro', label='class red')
plt.plot(x_blue[:,0], x_blue[:,1], 'bo', label='class blue')
plt.grid()
plt.legend(loc=2)
plt.xlabel('$x_1$', fontsize=15)
plt.ylabel('$x_2$', fontsize=15)
plt.axis([-4, 4, -4, 4])
plt.title('red vs. blue classes in the input space')
plt.show()

# Siec Neuronowa, a regresja logistyczna

Przypomnijmy sobie co robi regresja logistyczna. Z L7:

Przekształcając $$ \log(o) = \sum \theta_i x_i $$ otrzymujemy *bezpośrednio*, że $$ p(y | x) = \mbox{sigmoid}(\sum \theta_i x_i) $$, gdzie $sigmoid(a) = \frac{1}{1 + \exp(-a)}$.

Można ostatni wzór zapisać w postaci "sieci neuronowej":

<center><img width=500 src="figures/L9/logreg.png"></center>

To co pozostaje to jak wyznaczyć wagi przez optymalizacją. Przypomnijmy, że szukamy parametrów optymalizujących log-likelihood $$ LL(\hat y, y) = CE(\hat y, y) = \sum_{i=1}^{N} y \log\hat(y) $$. W regresji liniowej było prosto!

[Opisać/wyjaśnic cross entropy]

## Czego szuka regresja logistyczna

Regresja logistyczna dostaje kare za najdrobniejsza pomyłkę. Jeśli dla kasy 1 mówi z pewnością 99%, to dalej będzie niezerowy koszt:
    
<img width=500 src="figures/L9/logreg_vs_svm.png">

## Optymalizacja

Jak dotad nie rozpatrywaliśmy optymalizacji. Regresja liniowa ma rozwiązanie "zamknięte", ale nie wiemy jak optymalizować błąd regresji logistycznej!

$$ \theta^* = argmax_{\theta} L(p( \hat y | x, \theta), y) = CE(p(\hat y | x, \theta), y) = \sum_{i=1}^{N} y \log\hat(y) $$.

Dla klas binarnych:

$$ \sum_{i=1}^N y_i \log \hat y_i - (1 - y_i) \log (1 - \hat y_i ) $$

Jak znaleźć $\theta^*$? Użyjemy metody gradientowej:

$$ \theta^{t+1} = \theta^{t} - \alpha \frac{\partial L}{\partial \theta} $$

### Funkcja kosztu nie jest już taka prosta!

In [ ]:
# Plot the cost in function of the weights
# Define a vector of weights for which we want to plot the cost
nb_of_ws = 100 # compute the cost nb_of_ws times in each dimension
ws1 = np.linspace(-5, 5, num=nb_of_ws) # weight 1
ws2 = np.linspace(-5, 5, num=nb_of_ws) # weight 2
ws_x, ws_y = np.meshgrid(ws1, ws2) # generate grid
cost_ws = np.zeros((nb_of_ws, nb_of_ws)) # initialize cost matrix

# Fill the cost matrix for each combination of weights
for i in range(nb_of_ws):
    for j in range(nb_of_ws):
        cost_ws[i,j] = cost(nn(toy_train, np.asmatrix([ws_x[i,j], ws_y[i,j]])) , toy_y_train)
        
# Plot the cost function surface
plt.contourf(ws_x, ws_y, cost_ws, 20, cmap=cm.pink)
cbar = plt.colorbar()
cbar.ax.set_ylabel('$\\xi$', fontsize=15)
plt.xlabel('$w_1$', fontsize=15)
plt.ylabel('$w_2$', fontsize=15)
plt.title('Cost function surface')
plt.grid()
plt.show()

### Implementacja SGD

[Wyprowadzić na tablicy wzór na dLL/dtheta_i, jak to sie ma do backprop]

Jak właśnie pokazaliśmy $$ \frac{\partial L}{\partial \theta_i} = x_j (\hat y_i - t_i) $$

[Interpretacja geometryczna gradientu regresji logistycznej]

### Zadanie 1, Nauka regresji logistycznej przez SGD [2pkt]

1. Zaimplementuj SGD dla zbioru danych toy. 
2. Wygeneruj wizualizacje procesu uczenia (ostatnia komórka)
3. Narysuj wizualizacje dla learning_rate=0.05, 0.1 i 0.01
4. Dla jakiego learning_rate osiągamy najlepszą dokladnosc na (toy_test, toy_y_test)?

Co powinno wyjść dla odpowiednio 0.01 oraz 0.1:

<img width=200 src="figures/L9/lr0.01.png">

<img width=200 src="figures/L9/lr0.1.png">

In [ ]:
def sigmoid_activation(x):
    return 1.0 / (1 + np.exp(-x))

def gradient(w, x, t):     
    preds = sigmoid_activation(x.dot(w.T))
    y = np.array([n[0] for n in t])
    e = [n[0,0] for n in preds]
    error = e - y
 
    gradient = x.T.dot(error)
    return np.asmatrix(gradient)
    

def delta_w(w_k, x, t, learning_rate):
    return learning_rate * gradient(w_k,x,t)

# Parametry uczenia 
w = np.asmatrix([-4, -2])
learning_rate = 0.01

# Uczenie
nb_of_iterations = 10 
w_iter = [w] 
for i in range(nb_of_iterations):
    dw = delta_w(w, toy_train, toy_y_train, learning_rate) 
    w = w - dw 
    w_iter.append(w)

In [ ]:
# Plot the first weight updates on the error surface
# Plot the error surface
plt.contourf(ws_x, ws_y, cost_ws, 20, alpha=0.9, cmap=cm.pink)
cbar = plt.colorbar()
cbar.ax.set_ylabel('cost')

# Plot the updates
for i in range(1, 4): 
    w1 = w_iter[i-1]
    w2 = w_iter[i]
    # Plot the weight-cost value and the line that represents the update
    plt.plot(w1[0,0], w1[0,1], 'bo')  # Plot the weight cost value
    plt.plot([w1[0,0], w2[0,0]], [w1[0,1], w2[0,1]], 'b-')
    plt.text(w1[0,0]-0.2, w1[0,1]+0.4, '$w({})$'.format(i), color='b')
w1 = w_iter[3]  
# Plot the last weight
plt.plot(w1[0,0], w1[0,1], 'bo')
plt.text(w1[0,0]-0.2, w1[0,1]+0.4, '$w({})$'.format(4), color='b') 
# Show figure
plt.xlabel('$w_1$', fontsize=15)
plt.ylabel('$w_2$', fontsize=15)
plt.title('Gradient descent updates on cost surface')
plt.grid()
plt.show()

# keras

Sieci neuronowe definiujemy prawie tak prosto jak funkcje w Pythonie, a pochodne (to co liczyłem na tablicy) powinien nam liczyć komputer!

In [ ]:
dims = toy_train.shape[1]
print(dims, 'dims')
nb_classes = toy_y_train_one_hot.shape[1]
print(nb_classes, 'classes')

In [ ]:
# 1. Definicja modelu
model = Sequential()
model.add(Dense(nb_classes, input_shape=(dims,)))
model.add(Activation('softmax'))

# 2. Kompilacja
model.compile(optimizer=SGD(lr=0.1), loss='categorical_crossentropy')

# 3. Uczenie
model.fit(toy_train, toy_y_train_one_hot, epochs=10)

# 4. Predykcja
toy_y_pred = model.predict(toy_test).argmax(axis=1)
toy_y_train_pred = model.predict(toy_train).argmax(axis=1)
print model.layers[0].trainable_weights[0].get_value()
print np.mean(toy_y_train.reshape(-1,) == toy_y_train_pred)
print np.mean(toy_y_test.reshape(-1,) == toy_y_pred)

## Zadanie 2, Proste modyfikacje [2pkt]


1. (1pkt) Zmodyfikuj powyższą sieć tak aby osiągnęła 100% na zbiorze trenującym w rozważanym datasecie. Odpowiedz na pytania:
   * Czy ten model osiąga lepszy wynik na zbiorze testowym niż regresja logistyczna?
   * Czy można osiągnąć lepszy wynik na tym zbiorze danych, nawet jeśli posiadalibyśmy nieskończoną próbkę danych?
2. (1pkt) Naucz 2 warstwową sieć na smallMNIST. Czy dla jakiegoś rozmiaru ukrytej warstwy osiąga lepszy wynik niż regresja logistyczna

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(toy_train, toy_y_train)
Z = logreg.predict(toy_train)
round(np.mean(toy_y_train.reshape(-1,) == Z), 2)

In [ ]:
def find_the_best_model(test_lr=0.1, test_act='relu', test_nb_classes = [nb_classes], epochs=40, batch_size=10):    
    i = 30
    # 1. Definicja modelu
    model = Sequential()
    
    model.add(Dense(test_nb_classes[0], input_shape=(dims,), activation=test_act))
    for cl in test_nb_classes[1:-1]:
        model.add(Dense(cl, activation=test_act)) #, activation=test_act
    model.add(Dense(test_nb_classes[-1], activation='softmax'))

    # 2. Kompilacja
    model.compile(optimizer=SGD(lr=test_lr), loss='categorical_crossentropy')

    # 3. Uczenie
    model.fit(toy_train[:i], toy_y_train_one_hot[:i], epochs=epochs, verbose=0, batch_size=batch_size) #verbose=0,

    # 4. Predykcja
    toy_y_pred = model.predict(toy_test).argmax(axis=1)
    toy_y_train_pred = model.predict(toy_train[:i]).argmax(axis=1)
    #print model.layers[0].trainable_weights[0].get_value()
    
    if np.mean(toy_y_train.reshape(-1,) == toy_y_train_pred) > 0.8:
        print "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!"
    print round(np.mean(toy_y_train.reshape(-1,)[:i] == toy_y_train_pred), 2), '\t', test_lr, "\t", test_act, '\t', test_nb_classes
    
    #print np.mean(toy_y_test.reshape(-1,) == toy_y_pred)

def grid_search(grid):
    (keys, values_grid) = zip(*grid.iteritems())
    for values in product(*values_grid):
        yield dict(zip(keys, values))

In [ ]:
# lr = [0.01]
# act = ['relu']
# #cl = [[2,4,2,2], [3,4,2,2], [8,4,2,2], [20,4,2,2]]
# cl = [[90, 60, 30, 15, 2]]
# for d in grid_search({'lr': lr, 'act': act, 'cl': cl}):
#     find_the_best_model(test_lr=d['lr'], test_act=d['act'], test_nb_classes=d['cl'], epochs=5000, batch_size=500)

Najlepszy osiągnięty wynik:

0.97 	0.02 	relu 	[90, 60, 30, 15, 2]

a) Regresja logistyczna nie byłaby wstanie tak dobrze rozdzielić danych jak dobrze zaprojektowana sieć neuronowa.

b) Nie da się osiągnąć lepszego wyniku niż 100%. 

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(sm_train, sm_y_train)
print logreg.score(sm_test, sm_y_test)

In [ ]:
def find_the_best_model2(x, y, z, x_test, y_test, test_lr=0.1, test_act='relu', test_nb_classes = [nb_classes], epochs=40, batch_size=10):    
    # 1. Definicja modelu
    model = Sequential()

    model.add(Dense(test_nb_classes[0], input_shape=(test_nb_classes[0],), activation=test_act))
    for cl in test_nb_classes[1:-1]:
        model.add(Dense(cl, activation=test_act))
    model.add(Dense(test_nb_classes[-1], input_shape=(test_nb_classes[-1],), activation='softmax'))

    # 2. Kompilacja
    model.compile(optimizer=SGD(lr=1), loss='categorical_crossentropy',metrics=['acc'])

    # 3. Uczenie
    model.fit(sm_train, sm_y_train_one_hot, epochs=epochs, batch_size=batch_size)
    
    # 4. Predykcja
    y_pred = model.predict(x_test).argmax(axis=1)

    print round(np.mean(y_test == y_pred), 2), '\t', test_lr, "\t", test_act, '\t', test_nb_classes


In [ ]:
# sm_y_train_one_hot = keras.utils.to_categorical(sm_y_train)
# lr = [0.02]
# act = ['relu']
# cl = [[784, 10]]
# for d in grid_search({'lr': lr, 'act': act, 'cl': cl}):
#     find_the_best_model2(sm_train, sm_y_train, sm_y_train_one_hot, sm_test, sm_y_test, test_lr=d['lr'], test_act=d['act'], test_nb_classes=d['cl'], epochs=50, batch_size=10)

### Najlepszy wynik:

Epoch 1/50
500/500 [==============================] - 21s - loss: 2.3235 - acc: 0.1200
Epoch 2/50
500/500 [==============================] - 21s - loss: 1.3939 - acc: 0.5660
Epoch 3/50
500/500 [==============================] - 21s - loss: 2.1761 - acc: 0.4640
Epoch 4/50
500/500 [==============================] - 21s - loss: 2.7913 - acc: 0.4900
Epoch 5/50
500/500 [==============================] - 21s - loss: 1.7641 - acc: 0.4200
Epoch 6/50
500/500 [==============================] - 21s - loss: 1.1253 - acc: 0.7240
Epoch 7/50
500/500 [==============================] - 20s - loss: 0.6998 - acc: 0.8560
Epoch 8/50
500/500 [==============================] - 21s - loss: 0.5761 - acc: 0.8760
Epoch 9/50
500/500 [==============================] - 21s - loss: 0.5747 - acc: 0.8180
Epoch 10/50
500/500 [==============================] - 21s - loss: 0.7767 - acc: 0.7280
Epoch 11/50
500/500 [==============================] - 21s - loss: 0.9090 - acc: 0.7120
Epoch 12/50
500/500 [==============================] - 21s - loss: 0.7720 - acc: 0.7960
Epoch 13/50
500/500 [==============================] - 21s - loss: 0.5993 - acc: 0.8020
Epoch 14/50
500/500 [==============================] - 21s - loss: 0.3926 - acc: 0.9220
Epoch 15/50
500/500 [==============================] - 21s - loss: 0.2875 - acc: 0.9340
Epoch 16/50
500/500 [==============================] - 21s - loss: 0.2404 - acc: 0.9540
Epoch 17/50
500/500 [==============================] - 21s - loss: 0.2128 - acc: 0.9540
Epoch 18/50
500/500 [==============================] - 21s - loss: 0.1920 - acc: 0.9640
Epoch 19/50
500/500 [==============================] - 21s - loss: 0.1750 - acc: 0.9680
Epoch 20/50
500/500 [==============================] - 21s - loss: 0.1606 - acc: 0.9700
Epoch 21/50
500/500 [==============================] - 21s - loss: 0.1479 - acc: 0.9740
Epoch 22/50
500/500 [==============================] - 21s - loss: 0.1367 - acc: 0.9820
Epoch 23/50
500/500 [==============================] - 21s - loss: 0.1268 - acc: 0.9840
Epoch 24/50
500/500 [==============================] - 21s - loss: 0.1179 - acc: 0.9860
Epoch 25/50
500/500 [==============================] - 21s - loss: 0.1099 - acc: 0.9900
Epoch 26/50
500/500 [==============================] - 21s - loss: 0.1027 - acc: 0.9900
Epoch 27/50
500/500 [==============================] - 20s - loss: 0.0962 - acc: 0.9920
Epoch 28/50
500/500 [==============================] - 20s - loss: 0.0902 - acc: 0.9960
Epoch 29/50
500/500 [==============================] - 21s - loss: 0.0849 - acc: 0.9980
Epoch 30/50
500/500 [==============================] - 21s - loss: 0.0799 - acc: 0.9980
Epoch 31/50
500/500 [==============================] - 21s - loss: 0.0755 - acc: 0.9980
Epoch 32/50
500/500 [==============================] - 21s - loss: 0.0713 - acc: 1.0000
Epoch 33/50
500/500 [==============================] - 21s - loss: 0.0676 - acc: 1.0000
Epoch 34/50
500/500 [==============================] - 21s - loss: 0.0642 - acc: 1.0000
Epoch 35/50
500/500 [==============================] - 21s - loss: 0.0610 - acc: 1.0000
Epoch 36/50
500/500 [==============================] - 21s - loss: 0.0581 - acc: 1.0000
Epoch 37/50
500/500 [==============================] - 21s - loss: 0.0554 - acc: 1.0000
Epoch 38/50
500/500 [==============================] - 21s - loss: 0.0529 - acc: 1.0000
Epoch 39/50
500/500 [==============================] - 21s - loss: 0.0506 - acc: 1.0000
Epoch 40/50
500/500 [==============================] - 21s - loss: 0.0484 - acc: 1.0000
Epoch 41/50
500/500 [==============================] - 21s - loss: 0.0464 - acc: 1.0000
Epoch 42/50
500/500 [==============================] - 21s - loss: 0.0445 - acc: 1.0000
Epoch 43/50
500/500 [==============================] - 21s - loss: 0.0428 - acc: 1.0000
Epoch 44/50
500/500 [==============================] - 21s - loss: 0.0412 - acc: 1.0000
Epoch 45/50
500/500 [==============================] - 21s - loss: 0.0397 - acc: 1.0000
Epoch 46/50
500/500 [==============================] - 21s - loss: 0.0382 - acc: 1.0000
Epoch 47/50
500/500 [==============================] - 21s - loss: 0.0369 - acc: 1.0000
Epoch 48/50
500/500 [==============================] - 21s - loss: 0.0356 - acc: 1.0000
Epoch 49/50
500/500 [==============================] - 21s - loss: 0.0344 - acc: 1.0000
Epoch 50/50
500/500 [==============================] - 21s - loss: 0.0333 - acc: 1.0000

0.86 	0.02 	relu 	[784, 10]

Z tego wynika, że sieć osiąga lepsze wynika np. dla sieci o warstwach z neuronami 784, 10.

# Inne sieci neuronowe

Najprostsze co możemy zrobić to dodać warstwy. Po co omówimy w następnym rozdziale.

<img width=600 src="figures/L9/zoo.jpg">

# Znaczenie reprezentacji, kNN

Najproszym argumentem po co te sieci jest możliwość uczenia się lepszych reprezentacji niż pixele, czy ręczne cechy w zbiorze danych breast cancer. Omówimy to na przykładzie zbiorów danych: iris i MNIST oraz klasyczngo modelu kNN.

Uruchomimy najpierw model kNN na danych iris

<img width=600 src="https://sebastianraschka.com/images/blog/2015/principal_component_analysis_files/iris.png">

In [ ]:
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], test_size=0.15)

In [ ]:
plt.scatter(iris.data[iris['target']==0, 0], iris.data[iris['target']==0, 1], color='r')
plt.scatter(iris.data[iris['target']==1, 0], iris.data[iris['target']==1, 1], color='g')
plt.scatter(iris.data[iris['target']==2, 0], iris.data[iris['target']==2, 1], color='b')
plt.xlabel(iris['feature_names'][0])
plt.ylabel(iris['feature_names'][1])

In [ ]:
def knn_pred(x, x_train, y_train):
    current_best = -1, 100000
    for id in range(len(x_train)):
        dist = np.linalg.norm(x - x_train[id])
        if dist < current_best[1]:
            current_best = (id, dist)
    return y_train[current_best[0]]

## kNN na Iris

In [ ]:
Y_test_pred = []
for x in tqdm.tqdm(X_test, total=len(X_test)):
    Y_test_pred.append(knn_pred(x, X_train, y_train))
print str(int(100*np.mean(np.array(Y_test_pred) == y_test))),"%"

## small MNIST

Na Iris nasz prosty model osiąga 91%. Co sie dzieje na small MNIST?

In [ ]:
Y_test_pred = []
for x in tqdm.tqdm(sm_test, total=len(sm_test)):
    Y_test_pred.append(knn_pred(x, x_train=sm_train, y_train=sm_y_train))
print str(int(100*np.mean(np.array(Y_test_pred) == sm_y_test))),"%"

## Co sie stalo

In [ ]:
id_test = 15
plt.figure(figsize=(5, 5))
plt.suptitle("Pytanie")
plt.imshow(sm_test[id_test].reshape(28, 28), cmap=plt.cm.gray)

In [ ]:
all_dists = [np.linalg.norm(sm_test[id_test].reshape(-1,) - x) for x in sm_train]
all_dists = np.argsort(all_dists)

f, ax = plt.subplots(1, 3, figsize=(30, 10))

for id, i in enumerate(all_dists[0:3]):
    ax[id].imshow(sm_train[all_dists[id]].reshape(28, 28), cmap=plt.cm.gray)

## Nauka reprezentacji

Ref: http://colah.github.io/posts/2014-10-Visualizing-MNIST/

Czlowiek uczy się reprezentacji - widzimy na poziomie krawędzi i kształtów, a nie pixeli!

<img width=500 src="figures/L9/cat_experiment_no_reference.png">

Zastanówmy się wspólnie jaka reprezentacja byłaby fajna? Główna idea Representation Learning, może będzie o tym więcej wkrótce:

<img width=600 src="figures/L9/autoencoder_schema.jpg">

### PCA

Na obecną chwilę użyjemy PCA, o którym prawdopodobnie będzie dużo więcej w przyszłości. 

<img src="figures/L9/pca.png">

(ilustracja pochodzi z tutorialu na temat PCA oraz ICA w sklearn, http://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_vs_pca.html#sphx-glr-auto-examples-decomposition-plot-ica-vs-pca-py)

In [ ]:
pca = RandomizedPCA(n_components=8, iterated_power=15, whiten=True)
fig, plot = plt.subplots()
fig.set_size_inches(8, 8)
plt.prism()

X_transformed = pca.fit_transform(mnist_x_train.reshape(mnist_x_train.shape[0], -1))
plot.scatter(X_transformed[:, 0], X_transformed[:, 1], c=mnist_y_train)
plot.set_xticks(())
plot.set_yticks(())

In [ ]:
plt.figure(figsize=(20, 8 * 20))
plt.imshow(pca.components_.reshape((8 * 28, 28)).T, cmap="gray")

W przypadku twarzy PCA znajdzie bardzo ciekawy zestaw "cech", czy "wektorów bazowych":

<img width=400 src="figures/L9/eigenfaces.png">

## Zadanie 3, czy cechy pomagają w kNN [1pkt]

1. Naucz PCA jedynie na cyfrach 5 i załącz analogiczny plot jak powyżej

2. Użyj nauczonych cech (na całym zbiorze mnist) przez PCA do klasyfikacji kNN na zbiorze small MNIST. Jaki można osiągnąć wynik? Użyj KNeighborsClassifier z pakietu sklearn. Dopasuj parametry PCA oraz tak KNeighborsClassifier aby osiągnąć najlepszy wynik na zbiorze sm_test, wymagany jest wynik powyżej 80% na zbiorze testowym.

### 3.1

In [ ]:
idx = [idx for idx, val in enumerate(mnist_y_train) if val == 5]
mnist_x_train_5 = mnist_x_train[idx]
mnist_y_train_5 = mnist_y_train[idx]

In [ ]:
pca = RandomizedPCA(n_components=8, iterated_power=15, whiten=True)
fig, plot = plt.subplots()
fig.set_size_inches(8, 8)
plt.prism()
X_transformed = pca.fit_transform(mnist_x_train_5.reshape(mnist_x_train_5.shape[0], -1))

In [ ]:
plt.figure(figsize=(20,8* 20))
plt.imshow(pca.components_.reshape((8 * 28, 28)).T, cmap="gray")

### 3.2

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pca = RandomizedPCA(n_components=22, iterated_power=7, whiten=True)


X_transformed = pca.fit_transform(sm_train.reshape(sm_train.shape[0], -1))
knn = KNeighborsClassifier(n_neighbors=4, algorithm="kd_tree")
knn.fit(X_transformed, sm_y_train)
xtest = pca.transform(sm_test)
y_pred = knn.predict(xtest)
print metrics.accuracy_score(y_pred, sm_y_test)